# 安装 Redis

使用官方的 Docker 镜像安装 Redis.

- Docker Hub: [redis](https://hub.docker.com/_/redis/)
- GitHub: [redis/docker-library-redis](https://github.com/redis/docker-library-redis)

当前最新的版本是 7.4.0，考虑到我的五星级神机可能受不了太重的 Docker 环境，最终决定使用 [7.4.0-alpine](https://github.com/redis/docker-library-redis/blob/master/7.4/alpine/Dockerfile).

In [1]:
# !pip install redis

import time
import redis

import util

## 1. 下载镜像文件

来到 Dockerfile 所在目录，运行以下代码构建镜像：

```
# 从项目的根目录到 Dockerfile 所在目录
cd alpine

# 构建镜像
docker build -t redis-alpine-image .

# 查看所有 Docker 镜像
docker images
```

> 注意：我的机器 Dockerfile 必须加上 `RUN chmod +x /usr/local/bin/docker-entrypoint.sh` 才能运行成功，如果你用原版 Dockerfile 运行失败不妨试试。

## 2. 启动容器

一旦镜像构建完成，通过以下命令启动一个容器：

```
mkdir -p $(pwd)/../data
docker run --name some-redis \
    -p 6379:6379 \
    -v $(pwd)/../data:/data \
    -d redis-alpine-image \
    redis-server --save 60 1 --loglevel warning
```

其中：

- `--name some-redis`: 表示将当前容器命名为 `some-redis`
- `-p 6379:6379`: 将容器的 6379 端口映射到本机 6379 端口
- `-v $(pwd)/../data:/data`: 将容器的 `/data` 挂载到本地 `./data` 目录
- `-d`: 后台运行
- `redis-server --save 60 1 --loglevel warning`: 每 60 秒且至少有 1 次提交时做持久化，并将日志等级设为 warning

## 3. 交互式 shell

在容器内运行一个交互式 shell 会话：

```
docker exec -it some-redis /bin/sh
```

这可以帮助你了解容器的内部情况。在会话内运行：

```
redis-cli
ping
```

返回：`PONG`

## 4. 测试连接 

In [2]:
# 创建一个 Redis 连接
r = redis.Redis(host='localhost', port=6379, db=0)

# 测试连接
print(r.ping())

True


In [3]:
# 执行一些 Redis 命令
r.set('foo', 'bar')
print(r.get('foo'))

b'bar'


In [4]:
# 删除 foo
r.delete('foo')

1

## 5. 处理 CURD 逻辑

写一个自定义的 `util.RedisHandler` 类，来处理 Redis 的 CURD 逻辑。

In [5]:
rh = util.RedisHandler()
sleep_time = 5

print(f'rh.set: {rh.set("a", "1")}')
print(f'rh.set_with_expire: {rh.set_with_expire("b", "2", sleep_time)}')
print(f'rh.update: {rh.update("a", "3")}')
print(f'rh.get: {rh.get("a").decode("utf-8")}')

rh.set: True
rh.set_with_expire: True
rh.update: True
rh.get: 3


检查过期功能是否符合预期

In [6]:
print(f'rh.get: {rh.get("b")}')
print(f'sleep {sleep_time} seconds ...')
time.sleep(sleep_time)
print(f'rh.get: {rh.get("b")}')

rh.get: b'2'
sleep 5 seconds ...
rh.get: None
